In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\karth\AppData\Local\Temp\ipykernel_13528\840512378.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [3]:
from langchain_groq import ChatGroq
gK = os.getenv('GROQ_API_KEY')
llm = ChatGroq(model='Llama3-8b-8192',groq_api_key=gK)

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
import bs4

In [6]:
url='https://lilianweng.github.io/posts/2023-06-23-agent/'
loader = WebBaseLoader(web_path=(url),bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-content")),))
docs = loader.load()
print(type(docs),len(docs))

<class 'list'> 1


In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [8]:
import pprint

In [10]:
vectorstore = Chroma.from_documents(documents=splits,embedding=embeddings)
r = vectorstore.similarity_search('What is langchain?')
#pprint.pprint(r) # OK

In [11]:
retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    (
        "system","you are AI assistant" "\n\n{context}"
    ),
    (
        "human","{input}"
    )
])

# connect llm
#llm_obj = Ollama(model='gemma:2b')

qa_chain = create_stuff_documents_chain(llm,prompt)
ret_chain = create_retrieval_chain(retriver,qa_chain)

response = ret_chain.invoke({'input':'what is Self-Reflection'})
print(response['answer'])

C:\Users\karth\AppData\Roaming\Python\Python313\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Self-Reflection is a vital aspect of autonomous agents, allowing them to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a crucial role in real-world tasks where trial and error are inevitable.

In the context of the ReAct framework, Self-Reflection is the process of analyzing and learning from past actions and their outcomes. This involves reflecting on the thought process, actions taken, and observations made during a task or episode.

The ReAct framework uses a prompt template to guide this reflection, which includes explicit steps for the agent to think, act, and observe. The template is formatted as:

Thought: ...
Action: ...
Observation: ...
... (Repeated many times)

This process allows the agent to identify inefficiencies in its planning and correct hallucinations (sequences of identical actions that lead to the same observation). The agent's working memory stores up to three reflections, which are used as context for querying th

In [12]:
response = ret_chain.invoke({'input':'explain types of memory'})
print(response['answer'])

C:\Users\karth\AppData\Roaming\Python\Python313\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In the context of human brains, memory can be categorized into several types based on its characteristics, duration, and functionality. Here's a breakdown of the main types of memory:

1. **Sensory Memory**: This is the earliest stage of memory, providing the ability to retain impressions of sensory information (visual, auditory, tactile, etc.) after the original stimuli have ended. Sensory memory typically only lasts for up to a few seconds.

Subcategories of sensory memory include:

* **Iconic Memory** (visual): temporary storage of visual information, such as images or patterns.
* **Echoic Memory** (auditory): temporary storage of auditory information, such as sounds or melodies.
* **Haptic Memory** (touch): temporary storage of tactile information, such as touch or vibrations.

2. **Short-Term Memory (STM) or Working Memory**: This type of memory stores information that we are currently aware of and needed to carry out complex cognitive tasks such as learning and reasoning. Short-t

In [14]:
#pprint.pprint(r)

In [15]:
response = ret_chain.invoke({'input':'what is pddl?'})
print(response['answer'])

C:\Users\karth\AppData\Roaming\Python\Python313\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


PDDL stands for Planning Domain Definition Language. It is a formal language used to describe planning problems and domain knowledge. It was developed specifically for planning, which is the process of finding a sequence of actions to achieve a specific goal.

PDDL provides a common language for describing planning problems, making it possible for different planning systems and tools to communicate with each other. It is widely used in the field of artificial intelligence (AI) and robotics, particularly in areas such as:

1. Planning: PDDL is used to describe planning problems, including the initial state, goals, and constraints.
2. Robotics: PDDL is used to describe the planning problems for robots, including the robot's capabilities, the environment, and the goals.
3. Artificial Intelligence: PDDL is used in AI research to describe planning problems and test planning algorithms.

PDDL has two main components:

1. Domain Description: This component describes the planning domain, inclu